In [10]:
import cv2
import os

def extract_frames_opencv(video_path, output_dir="frames", fps_interval=1):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps * fps_interval)

    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{saved_count:04d}.png")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"Done: {saved_count} frames saved to {output_dir}")

# Example usage
extract_frames_opencv(
    r"C:\Users\KartheekMakkena\Downloads\videoplayback.webm",
    r"C:\Users\KartheekMakkena\OneDrive - Hanyaa\Desktop\New folder (3)",
    fps_interval= 0.5 # 1 frame every 1 second
)

Done: 89 frames saved to C:\Users\KartheekMakkena\OneDrive - Hanyaa\Desktop\New folder (3)


In [11]:
import os
from ollama import chat
import re

# Set the folder path
folder_path = r"C:\Users\KartheekMakkena\OneDrive - Hanyaa\Desktop\New folder (3)"

def extract_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else float('inf')

image_paths = sorted(
    [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.png')],
    key=lambda x: extract_number(os.path.basename(x))
)
content=""
batch_size = 10
for i in range(0, len(image_paths)-len(image_paths)%batch_size, batch_size):
    batch = image_paths[i:i + batch_size]
    
    response = chat(
        model='gemma3:27b',
        messages=[
            {
                'role': 'user',
                'content': f"give 1 line discriptions for images such that those contain each and every detail({i+1}-{i+len(batch)}). Be descriptive.",
                'images': batch,
            }
        ],
    )
    
    print(f"\n--- Response for images {i+1} to {i+len(batch)} ---")
    print(response['message']['content'])
    
    summary = chat(model='gemma3:27b',
        messages=[
            {
                'role': 'user',
                'content': f"summarize in 3 lines swch that each details in the images should be covered{response['message']['content']}",
                'images': batch,
            }
        ],
    )
    content=content+f"""Response for images {i+1} to {i+len(batch)}\n{summary['message']['content']}\n"""


--- Response for images 1 to 10 ---
Here are one-line descriptions for each of the images, aiming for detail:

1. **Image 1:** A vibrant cartoon scene depicts the title "The Hare and The Tortoise" in bold red letters above a hare and tortoise standing on a checkered track within a lush green forest.
2. **Image 2:** A cartoon image features a playful hare and a determined tortoise poised at a checkered start line in a forest clearing, with a red "START" flag marking the beginning of their race.
3. **Image 3:** The scene showcases the hare and tortoise ready to race amidst a dense forest, a checkered track laying beneath them, and a "START" flag signaling the race’s commencement, with the title “The Hare and The Tortoise” appearing above.
4. **Image 4:** The image shows the hare and tortoise standing on a checkered path at the edge of a forest, with a "START" flag and the title “The Hare and The Tortoise” prominently displayed.
5. **Image 5:** A vibrant illustration features a hare and 

In [15]:
print(content)

Response for images 1 to 10
Here's a 3-line summary covering the details of the images:

These vibrant cartoon images illustrate the classic fable of "The Hare and The Tortoise," depicting the two characters poised for a race. They are set within a lush forest environment, marked by a checkered race track and a red "START" flag signaling the beginning of their contest.  The scenes capture the anticipation of the race, with some images introducing the narrative with the phrase "One day."
Response for images 11 to 20
Here's a 3-line summary covering the details in the images:

A cheerful purple hare and a calm green tortoise are poised at the starting line of a race, indicated by a red "START" flag. 
The racecourse is marked by a black and white checkered pattern, set within a lush, vibrant green forest filled with tall trees.
The scene exudes a playful, cartoonish atmosphere, suggesting a classic tale about to unfold between the two unlikely competitors.
Response for images 21 to 30
Her

In [14]:
kar=f""" this the information from the frames extracted from the summarize overall {content}"""
model=chat(model='gemma3:27b',
    messages=[{
        'role': 'user',
        'content': kar,
    }]
)

print(model['message']['content'])

## Overall Summary of the Image Sequence (Images 1-80)

This image sequence vividly illustrates the classic fable of "The Tortoise and the Hare." The story begins with a confident hare and a determined tortoise embarking on a race through a vibrant forest. The hare quickly takes a substantial lead, becoming overconfident and taking a nap while the tortoise steadily continues. As the hare sleeps, the tortoise perseveres, eventually overtaking him and winning the race. The sequence culminates in the tortoise triumphantly receiving a trophy, reinforcing the moral of the story: slow and steady wins the race. The images are bright, cartoonish, and effectively convey the narrative arc of this beloved tale.




